In [1]:
from dask.distributed import Client, LocalCluster

# Example: 4 workers, 1 thread each
cluster = LocalCluster(n_workers=4, threads_per_worker=1)
client = Client(cluster)

print("Dashboard:", client.dashboard_link)


Dashboard: http://127.0.0.1:8787/status


In [ ]:
# CLUSTER DEPLOYMENT ON CLOUDVENETO
from dask.distributed import Client, SSHCluster

cluster = SSHCluster(
    ["10.67.22.154", "10.67.22.216", "10.67.22.116", "10.67.22.113"],
    connect_options={"known_hosts": None},
    remote_python="/home/ubuntu/miniconda3/bin/python",
    scheduler_options={"port": 8786, "dashboard_address": ":8797"},
    worker_options={
        "nprocs": 1,       # N. of processess per VM. CloudVeneto's large VM offers 4-core CPU, but for now we only spawn 1 process per VM
        "nthreads": 1      # N. of threads per process
    }
)

client = Client(cluster)


In [2]:
# check if everything went smoothly
print(client)
print(cluster)


<Client: 'tcp://127.0.0.1:39081' processes=4 threads=4, memory=5.79 GiB>
LocalCluster(8cdcf355, 'tcp://127.0.0.1:39081', workers=4, threads=4, memory=5.79 GiB)


In [3]:
import dask.array as da
import dask
import numpy as np
import time
from scipy.linalg import solve_triangular 
from sklearn.datasets import fetch_california_housing

# Download California Housing dataset
data = fetch_california_housing(as_frame=True)

# Convert features into Dask Array (it's a matrix).
n_partition = 3        # number of partition in memory. We have 4 VMS (1 master + 3 workers), so let's start with just 3 partitions
length_partition = data.data.shape[0] // n_partition
X_da = da.from_array(data.data.values, chunks=(length_partition, data.data.shape[1]))

print("Number of Dask partitions:",  X_da.npartitions) 
print("Length of each partition:", length_partition, "rows")
print("Length of the whole dataset:", data.data.shape[0], "rows")


Number of Dask partitions: 3
Length of each partition: 6880 rows
Length of the whole dataset: 20640 rows


In [4]:
X_cached = X_da.rechunk((1_000_000, -1)).persist()

print(X_da)


dask.array<array, shape=(20640, 8), dtype=float64, chunksize=(6880, 8), chunktype=numpy.ndarray>


In [7]:
def indirect_serial(A, n_div):

    """
    Indirect TSQR (serial, NumPy).
    Splits A by rows into n_div blocks, computes local R_i via QR,
    reduces to global R by QR on the stacked R_i, then recovers Q = A R^{-1}.
    Returns (Q, R).
    """

    n_samp = A.shape[0]
    
    div_points = int(np.floor(n_samp/n_div))
    A_divided = []
    Ri = []
    
    A_divided = [A[div_points * i : div_points * (i + 1)] for i in range(n_div - 1)]
    A_divided.append(A[(n_div - 1) * div_points:, :])   # In the case n_samp wasn't divisible by n_div

    Ri = [np.linalg.qr(Ai, mode="reduced")[1] for Ai in A_divided]
    
    R_stack = np.concatenate(Ri, axis = 0)

    # Step 3. Stack R_i and compute global R

    _, R = np.linalg.qr(R_stack, mode="reduced")


    #   I = np.eye(n_samp, dtype=A.dtype)
    #   Rinv = solve_triangular(R, I, lower=False)
    Rinv = np.linalg.inv(R)
    Q = A @ Rinv

    return Q, R


def compute_R(block):
    # np.linalg.qr with mode='r' gives just the R matrix
    R = np.linalg.qr(block, mode="r")
    return R


def indirect_parallel(X_da):

    """
    Indirect TSQR with Dask.
    Input:
        X_da : Dask Array (m x n), chunked row-wise
    Output:
        R    : final global triangular factor (n x n, NumPy array on driver)
        Q_da : Dask Array (m x n), representing Q = A R^{-1} (lazy)
    """


    n_cols = X_da.shape[1]

    R_blocks = X_da.map_blocks(compute_R, dtype=X_da.dtype, chunks=(n_cols, n_cols))
    # Now R_blocks is a stack of n x n matrices (one per partition)
    # Its shape is (#chunks * n, n)

    R_blocks.visualize(filename="fig/R_blocks_graph", format="png")
 

    #Dask has da.linalg.qr, but it assumes the whole array is large and chunked regularly.
    #To get the final global R, you must combine all the Ri
    #That means at some point, the data has to come together into a single place (can’t keep it sharded).
    # So we bring the data to the driver because it is very small, because it optimizes the uses of np.linalg.qr, we are gathering the small stuff
    R_stack = R_blocks.compute()   # NumPy array, shape (p*n, n)

    # Small QR on driver to combine them into the final R
    R = np.linalg.qr(R_stack, mode="r")
    # delay the computing of qr

    # Instead of materializing Q, compute a small R^{-1} (n x n).
    I = np.eye(n_cols, dtype=X_da.dtype)
    R_inv = solve_triangular(R, I, lower=False)  # stable

    # Broadcast Rinv to every chunk: Q = A @ R^{-1}
    Q_da = X_da @ R_inv   # still a Dask Array, lazy

    return Q_da, R      #Q_da because it is lazy, it is still a Dask array


def indirect_parallel_optimized(X_da):


    n_cols = X_da.shape[1]

    R_blocks = X_da.map_blocks(compute_R, dtype=X_da.dtype, chunks=(n_cols, n_cols))
    # Now R_blocks is a stack of n x n matrices (one per partition)
    # Its shape is (#chunks * n, n)

    R_blocks.visualize(filename="fig/R_blocks_graph", format="png")
 

    #Dask has da.linalg.qr, but it assumes the whole array is large and chunked regularly.
    #To get the final global R, you must combine all the Ri
    #That means at some point, the data has to come together into a single place (can’t keep it sharded).
    # So we bring the data to the driver because it is very small, because it optimizes the uses of np.linalg.qr, we are gathering the small stuff
    R_stack = R_blocks.compute()   # NumPy array, shape (p*n, n)

    # Small QR on driver to combine them into the final R
    R = np.linalg.qr(R_stack, mode="r")
    # delay the computing of qr

    # Instead of materializing Q, compute a small R^{-1} (n x n).
    I = np.eye(n_cols, dtype=X_da.dtype)
    R_inv = solve_triangular(R, I, lower=False)  # stable

    Rinv_da = da.from_array(R_inv, chunks=(R_inv.shape[0], R_inv.shape[1]))
    Q_da = X_da @ Rinv_da

    return Q_da, R      #Q_da because it is lazy, it is still a Dask array



def indirect_parallel_partially_delayed(X_da):

    n_cols = X_da.shape[1]

    R_blocks = X_da.map_blocks(compute_R, dtype=X_da.dtype, chunks=(n_cols, n_cols))
    # Now R_blocks is a stack of n x n matrices (one per partition)
    # Its shape is (#chunks * n, n)

    R_blocks.visualize(filename="fig/R_blocks_graph", format="png")

    # 2) Convert blocks to delayed NumPy arrays, stack via delayed
    R_list = list(R_blocks.to_delayed().ravel())     # each is delayed np.ndarray (n x n)
    R_stack = dask.delayed(np.vstack)(R_list)        # delayed (p*n x n)


    R_da = dask.delayed(compute_R)(R_stack)      # delayed np.ndarray (n x n)

    # 4) Materialize small R on driver; solve for R^{-1}
    R = R_da.compute()                           # NumPy (n x n)
    I = np.eye(n_cols, dtype=X_da.dtype)
    R_inv = solve_triangular(R, I, lower=False)       # NumPy (n x n)

    # 5) Broadcast multiply (keep Q lazy)
    Q_da = X_da @ R_inv

    return Q_da, R      #Q_da because it is lazy, it is still a Dask array




def indirect_parallel_delayed(X_da):

    n_cols = X_da.shape[1]

    R_blocks = X_da.map_blocks(compute_R, dtype=X_da.dtype, chunks=(n_cols, n_cols))
    # Now R_blocks is a stack of n x n matrices (one per partition)
    # Its shape is (#chunks * n, n)


    # 2) Convert blocks to delayed NumPy arrays, stack via delayed
    R_list = list(R_blocks.to_delayed().ravel())     # each is delayed np.ndarray (n x n)
    R_stack = dask.delayed(np.vstack)(R_list)        # delayed (p*n x n)
    

    R_delayed = dask.delayed(compute_R)(R_stack)      # delayed np.ndarray (n x n)

    # 3) Turn the small delayed R into a Dask Array (single (n,n) chunk)
    R_da = da.from_delayed(R_delayed, shape=(n_cols, n_cols), dtype=X_da.dtype)

    # 4) Materialize small R on driver; solve for R^{-1}
    I = np.eye(n_cols, dtype=X_da.dtype)

    # compute R^{-1} lazily
    R_inv_delayed = dask.delayed(solve_triangular)(R_delayed, I, lower=False)
    R_inv_da = da.from_delayed(R_inv_delayed, shape=(n_cols, n_cols), dtype=X_da.dtype)

    # 5) Broadcast multiply (keep Q lazy)
    Q_da = X_da @ R_inv_da

    return Q_da, R_da      #Q_da because it is lazy, it is still a Dask array


In [8]:
%%time

Q, R = indirect_serial(data.data.values, 50)

size = 20640 / 50 * 8 * 8 #rows_per_chunk × n_cols × 8 bytes
print("The size for each chunk is :", size/1e6, "Mb")


The size for each chunk is : 0.0264192 Mb
CPU times: user 95.4 ms, sys: 0 ns, total: 95.4 ms
Wall time: 36.5 ms


In [9]:
%%time

Q_da, R = indirect_parallel(X_da)   # X_da is already splitted into three partitions


print("Final R shape:", R.shape)  # (n, n), small
print("Q is lazy:", Q_da)         # Dask Array, not yet computed

print("Chunks:", X_da.chunks)
print("Number of partitions:", X_da.npartitions)
print("Total size [MB]:", X_da.nbytes / 1e6)
print("Chunk size [MB]:", X_da.nbytes / X_da.npartitions / 1e6)    # 10–100 MB

print("Shape:", X_da.shape)
print("Chunks:", X_da.chunks)
print("Partitions:", X_da.npartitions)
print("Total size [MB]:", X_da.nbytes/1e6)
print("Avg chunk [MB]:", (X_da.nbytes/X_da.npartitions)/1e6)


#print("Compact sanity check:", X_da.shape, "(m,n);", R_blocks.shape, "(n *n_partitions, n);", R.shape, "(n,n)" )


Final R shape: (8, 8)
Q is lazy: dask.array<getitem, shape=(20640, 8), dtype=float64, chunksize=(6880, 8), chunktype=numpy.ndarray>
Chunks: ((6880, 6880, 6880), (8,))
Number of partitions: 3
Total size [MB]: 1.32096
Chunk size [MB]: 0.44032
Shape: (20640, 8)
Chunks: ((6880, 6880, 6880), (8,))
Partitions: 3
Total size [MB]: 1.32096
Avg chunk [MB]: 0.44032
CPU times: user 162 ms, sys: 0 ns, total: 162 ms
Wall time: 851 ms


In [10]:
# not-delayed version
start = time.perf_counter()
Q_nd, R_nd = indirect_parallel(X_cached)  # returns Q_da, R (NumPy)
_ = R_nd  # already materialized
t_nd_R = time.perf_counter() - start

# optimized not-delayed version
start = time.perf_counter()
Q_po, R_po = indirect_parallel_optimized(X_cached)  # returns Q_da (Dask), R_da (Dask)
_ = R_po  # already materialized
t_po_R = time.perf_counter() - start

# partially-delayed version
start = time.perf_counter()
Q_pd, R_pd = indirect_parallel_partially_delayed(X_cached)  # returns Q_da (Dask), R (NumPy)
_ = R_pd  # already materialized inside the function
t_pd_R = time.perf_counter() - start

# fully-delayed version
start = time.perf_counter()
Q_fd, R_fd = indirect_parallel_delayed(X_cached)  # returns Q_da (Dask), R_da (Dask)
_ = R_fd.compute()  # FORCE the same final R compute
t_fd_R = time.perf_counter() - start


print(f"R time — not-delayed: {t_nd_R:.3f}s | optimized: {t_po_R:.3f}s | partial: {t_pd_R:.3f}s | full-delayed: {t_fd_R:.3f}s")


R time — not-delayed: 0.107s | optimized: 0.066s | partial: 0.052s | full-delayed: 0.032s


In [14]:
# e.g., apply Q to a random vector and compute the norm — same workload for all
v = np.random.randn(X_cached.shape[1]).astype(X_cached.dtype)

def time_Q_apply(Q_da, v):
    start = time.perf_counter()
    val = da.linalg.norm(Q_da @ v).compute()  # triggers the matmul + reduction
    end = time.perf_counter()
    return end - start, val

t_nd_Q, val_nd = time_Q_apply(Q_nd, v)
t_po_Q, val_po = time_Q_apply(Q_po, v)
t_pd_Q, val_pd = time_Q_apply(Q_pd, v)
t_fd_Q, val_fd = time_Q_apply(Q_fd, v)

print(f"Q apply — not-delayed: {t_nd_Q:.3f}s | partial: {t_po_Q:.3f}s | partial: {t_pd_Q:.3f}s | full-delayed: {t_fd_Q:.3f}s")


Q apply — not-delayed: 0.080s | partial: 0.057s | partial: 0.047s | full-delayed: 0.056s


In [ ]:
N_WORKERS = 3
# Initialization of a distributed random matrix
m, n = int(1e7), 4
chunks = [m // N_WORKERS for _ in range(N_WORKERS-1)]
chunks.append(m - sum(chunks))
A = da.random.random((m, n), chunks=(chunks, n))

# Persist in memory to avoid recomputation
A = A.persist() 

print(f"Input matrix A: m = {A.shape[0]}, n = {A.shape[1]}")
print(f"The {len(A.chunks[0])} blocks are: {A.chunks[0]}")
print(f"Total size of A: {A.nbytes / 1e6} MB")
A


Input matrix A: m = 10000000, n = 4
The 3 blocks are: (3333333, 3333333, 3333334)
Total size of A: 320.0 MB


dask.array<random_sample, shape=(10000000, 4), dtype=float64, chunksize=(3333334, 4), chunktype=numpy.ndarray>

In [ ]:
# QR decomposition

Q_delayed, R_delayed = indirect_parallel_delayed(A)  # Our implementation
Q_delayed_dask, R_delayed_dask = da.linalg.tsqr(A)    # Dask's implementation

print("-- OUR IMPLEMENTATION (QR) --")
%time Q, R = dask.compute(Q_delayed, R_delayed)

print("\n-- DASK'S IMPLEMENTATION (QR) --")
%time Q_dask, R_dask = dask.compute(Q_delayed_dask, R_delayed_dask)


-- OUR IMPLEMENTATION (QR) --
CPU times: user 14.2 s, sys: 5.63 s, total: 19.8 s
Wall time: 19.9 s

-- DASK'S IMPLEMENTATION (QR) --
CPU times: user 1.37 s, sys: 1.86 s, total: 3.23 s
Wall time: 16.6 s


In [ ]:
# create again a cluster
# CLUSTER DEPLOYMENT ON CLOUDVENETO
client.close()   # close the previous one
cluster.close()  # close the previous one

cluster = SSHCluster(
    ["10.67.22.154", "10.67.22.216", "10.67.22.116", "10.67.22.113"],
    connect_options={"known_hosts": None},
    remote_python="/home/ubuntu/miniconda3/bin/python",
    scheduler_options={"port": 8786, "dashboard_address": ":8797"},
    worker_options={
        "nprocs": 4,        # Now we use all 4 cores -> 12 workers
        "nthreads": 1       # We use 1 threads. Following Dask documentation, however, Numpy should release well the GIL lock thus we could use more than 1 thread. 
    }
)

client = Client(cluster)


In [ ]:
print(client)
print(cluster)


In [ ]:
cluster


In [ ]:
import dask.dataframe as dd
import os

os.chdir("/home/ubuntu") 
path_HIGGS = os.getcwd() + "/datasets/HIGGS.csv"
# A huge dataset
df = dd.read_csv(path_HIGGS, header=None, blocksize="200MB")    # The block size can be customized, let's start with 200 MB
X_df = df.iloc[:, 1:] 
X_da = X_df.to_dask_array(lengths=True)


In [1]:
#Let's print it
X_da


NameError: name 'X_da' is not defined

In [ ]:

# Print the number of partition
X_da.npartitions


In [ ]:

%%time

Q_da, R = indirect_parallel(X_da)


In [ ]:
%%time
R = indirect_parallelR(X_da)


In [ ]:
# Rechunk to ~1M rows per block (~100 MB) so each worker gets a good-sized task
X_da_cached = X_da.rechunk((1_000_000, -1)).persist()

# First persist actually loads CSV into memory on workers
# This itself will take a few seconds (same cost as run 1)


In [ ]:
%%time
Q_da2, R2 = indirect_parallel(X_da_cached)   # Now works from cached chunks in RAM


In [ ]:
%%time
R = indirect_parallelR(X_da_cached)
